In [12]:
from numpy import size
import gc
import random as random
%env XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/lib/cuda
import tensorflow as tf
from scripts import commonUtils as alex
from modelli import modelli as modelli

# Allow memory growth for the GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
seed=random.randint(1,99999)
size=(100,100)
dataPath='images100'
pool=alex.getSamples(dataPath,size,batchSize=99999,seed=seed,split=0.2)
tf.keras.backend.clear_session()
poolValidation=alex.getSamples(dataPath,batchSize=9999,imgSize=size,seed=seed,training=False,split=0.2)

poolValidation.restPool()
valPool=poolValidation.getBatch(reshae=False)
batch=pool.getBatch(reshae=False)


env: XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/lib/cuda
Found 8108 files belonging to 1 classes.
Using 6487 files for training.
Found 8108 files belonging to 1 classes.
Using 1621 files for validation.
1621   1621
got another batch,  0  remaning  we are using  1621
6487   6487
got another batch,  0  remaning  we are using  6487


In [2]:
autoencoder=tf.keras.models.load_model("modelli/BN-RGB-50-NO-RETRAIN",{"ssim_loss":modelli.ssim_loss,"costum_loss":modelli.costum_loss})
model=modelli.AutoencoderFinal(modelli.Autoencoder())
model(batch[:5])
model.load_weights('autoencoder_final_weights.h5')


2025-01-27 13:01:33.800004: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902


In [3]:
class MyModel(tf.keras.Model):

  def __init__(self,decoder):
    super(MyModel, self).__init__()
    self.decoder = decoder

  def call(self, inputs):
    output=self.decoder(inputs)
    return output

In [5]:
import io
import numpy as np
from PIL import Image

modelF=MyModel(model.decoder)
modelF.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                       loss=modelli.costum_loss,
                       metrics=['mse',modelli.ssim_loss,modelli.costum_loss])
results = []
output=[]
size=256
for k in  range(0,25):
     residuals=model.encoder(batch[(k)*size:(k+1)*size])
     for i,res in enumerate(residuals):
        residuals[i]=res.numpy()
     for i in range(0,size):
            for j,res in enumerate(residuals):
                if(res.shape[3]==1):
                     image=Image.fromarray(np.squeeze(res[i]), mode='L')
                else:
                     image=Image.fromarray(res[i])
                buffer = io.BytesIO()
                buffer.tell()
                image.save(buffer, format="JPEG", quality=90)
                imageCompressed=Image.open(buffer)
                if(res.shape[3]==1):
                     residuals[j][i]=tf.expand_dims(imageCompressed,axis=-1)
                else:
                     residuals[j][i]=imageCompressed
                buffer.tell()
     if len(output)==0:
          output=model.converter(residuals)
     else:
          tensors=model.converter(residuals)
          output[0]=tf.concat([output[0], tensors[0]], axis=0)
          output[1]=tf.concat([output[1], tensors[1]], axis=0)
     
modelF.fit(output, batch[0:(25)*size], # Note!
                  epochs=150,
                  shuffle=True,
                  batch_size=256,
                  validation_data=(output, batch[0:(25)*size]),
               callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=30)])

Epoch 1/150


2025-01-27 13:02:08.055902: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f7d3fe7fc10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-01-27 13:02:08.055929: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-01-27 13:02:08.059021: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-01-27 13:02:08.124360: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


25/25 [==============================] - 10s 152ms/step - loss: 0.0050 - mse: 0.0040 - ssim_loss: 0.2557 - costum_loss: 0.0050 - val_loss: 0.0047 - val_mse: 0.0038 - val_ssim_loss: 0.2433 - val_costum_loss: 0.0047
Epoch 2/150
25/25 [==============================] - 2s 74ms/step - loss: 0.0046 - mse: 0.0037 - ssim_loss: 0.2496 - costum_loss: 0.0046 - val_loss: 0.0046 - val_mse: 0.0037 - val_ssim_loss: 0.2444 - val_costum_loss: 0.0046
Epoch 3/150
25/25 [==============================] - 2s 77ms/step - loss: 0.0046 - mse: 0.0037 - ssim_loss: 0.2524 - costum_loss: 0.0046 - val_loss: 0.0046 - val_mse: 0.0037 - val_ssim_loss: 0.2443 - val_costum_loss: 0.0046
Epoch 4/150
25/25 [==============================] - 2s 75ms/step - loss: 0.0045 - mse: 0.0036 - ssim_loss: 0.2460 - costum_loss: 0.0045 - val_loss: 0.0045 - val_mse: 0.0036 - val_ssim_loss: 0.2426 - val_costum_loss: 0.0045
Epoch 5/150
25/25 [==============================] - 2s 73ms/step - loss: 0.0045 - mse: 0.0036 - ssim_loss: 0.2437

KeyboardInterrupt: 